
## Projet : Optimisation de Tournées de Livraison pour l'ADEME

# Livrable final 

**Groupe** : Groupe 2

**Membres** :
- ABDELKKADERMEKKI Mohamed 
- BENAZIZ Rayan
- HALLAOUA Sid-Ali
- RECHAM Wissam


---

## Table des Matières

1. [Rappel du Contexte et des Objectifs](#1-Rappel-du-Contexte-et-des-Objectifs)

2. [Modélisation Mathématique Formelle](#3-modélisation-mathématique-formelle)

3. [Méthode de Résolution Choisie](#4-Méthode-de-Résolution-Choisie)

4. [Implémentation](#4-Implémentation)

5. [Etude expérimentale](#5-Etude-expérimentale)

6. [Conclusion](#6-conclusion)



---

# Rappel du Contexte et des Objectifs

Dans le cadre de l’appel à manifestation d’intérêt lancé par l’ADEME, l’objectif général est de proposer des solutions innovantes permettant d’optimiser la mobilité et la logistique dans différents types de territoires.
Les enjeux actuels de mobilité — hausse de la demande de livraison, contraintes environnementales, variabilité du trafic, optimisation des ressources — rendent indispensable l’utilisation de méthodes de Recherche Opérationnelle pour améliorer l’efficacité des tournées de transport.

Notre équipe a choisi de se concentrer sur l’optimisation des tournées de livraison, un problème central en logistique.
Dans ce contexte, nous étudions une variante avancée du problème classique de tournées : le Vehicle Routing Problem with Time Windows (VRPTW).

Le VRPTW consiste à déterminer un ensemble de tournées permettant de desservir un ensemble de clients à partir d’un dépôt unique, en minimisant le coût total (généralement la distance ou la durée) tout en respectant des fenêtres temporelles strictes.
Chaque client doit être servi dans un créneau précis, ce qui rend le problème sensiblement plus complexe que le VRP classique.

Ce projet poursuit trois objectifs principaux :
 1. Proposer un modèle formel robuste.
 2. Concevoir et implémenter un algorithme performant.
 3. Évaluer expérimentalement les performances.

# Modèle Mathématique du CVRPTW selon l'implémentation ALNS

## 1. Présentation du Problème

Le **CVRPTW** (Capacitated Vehicle Routing Problem with Time Windows) est un problème d'optimisation combinatoire consistant à déterminer un ensemble de tournées de véhicules pour desservir des clients, en minimisant le coût total tout en respectant :
- Les **contraintes de capacité** des véhicules
- Les **fenêtres temporelles** de livraison de chaque client
- Les **temps de service** à chaque point de livraison


## 2. Représentation par Graphe

Le problème est modélisé sur un **graphe complet orienté** : $G = (V, A)$

### Ensemble des nœuds
$$V = \{0, 1, 2, ..., n\}$$

Où :
- $0$ : Le **dépôt** (point de départ et d'arrivée de tous les véhicules)
- $i \in \{1, ..., n\}$ : Les $n$ **clients** à desservir

### Ensemble des arcs
$$A = \{(i,j) : i, j \in V, i \neq j\}$$

Chaque arc $(i,j)$ représente un trajet possible du nœud $i$ vers le nœud $j$.

**Propriétés :**
- Graphe **complet** : il existe un arc entre chaque paire de nœuds
- Graphe **orienté** : nécessaire pour la modélisation des contraintes temporelles
- Graphe **pondéré** : chaque arc a un poids $d_{ij}$ (distance ou temps)


## 3. Paramètres du Problème

<u>3.1 Paramètres géographiques</u>

| Notation | Description | Type |
|----------|-------------|------|
| $n$ | Nombre de clients à desservir | Entier |
| $d_{ij}$ | Temps de trajet entre les nœuds $i$ et $j$ | Réel $\geq 0$ |

**Propriétés de la matrice de temps :**
- $d_{ii} = 0$ pour tout $i$ (temps nul d'un point vers lui-même)
- Symétrie (distances euclidiennes) : $d_{ij} = d_{ji}$
- Inégalité triangulaire : $d_{ij} \leq d_{ik} + d_{kj}$, $\forall i,j,k \in V$

<u>3.2 Paramètres des clients</u>

Pour chaque client $i \in V$ :

| Notation | Description | Unité |
|----------|-------------|-------|
| $q_i$ | Demande du client $i$ (quantité à livrer) | kg, unités |
| $s_i$ | Temps de service au client $i$ | minutes |
| $a_i$ | Début de la fenêtre temporelle (earliest time) | minutes |
| $b_i$ | Fin de la fenêtre temporelle (latest time) | minutes |

**Convention pour le dépôt** (nœud 0) :
- $q_0 = 0$ (pas de demande)
- $s_0 = 0$ (pas de temps de service)
- $[a_0, b_0]$ : Horaires d'ouverture du dépôt

<u>3.3 Paramètres des véhicules</u>

| Notation | Description | Type |
|----------|-------------|------|
| $K$ | Nombre de véhicules disponibles | Entier |
| $Q$ | Capacité maximale d'un véhicule | kg, unités |

**Hypothèses :**
- Flotte **homogène** : tous les véhicules ont la même capacité $Q$
- Dépôt **unique** : tous les véhicules partent du même point
- Nombre de véhicules **suffisant** pour desservir tous les clients


## 4. Variables de Décision

<u>4.1 Formulation exacte (programmation linéaire)</u>

**Variables binaires de routage :**
$$x_{ijk} \in \{0, 1\}, \quad \forall (i,j) \in A, \forall k \in K$$

Définition :
$$x_{ijk} = \begin{cases} 
1 & \text{si le véhicule } k \text{ emprunte l'arc } (i,j) \\
0 & \text{sinon}
\end{cases}$$

**Variables temporelles :**
$$t_i \in \mathbb{R}^+, \quad \forall i \in V$$

Où $t_i$ représente le **temps d'arrivée** (ou début de service) au nœud $i$.

**Variables auxiliaires (MTZ) :**
$$u_i \in \mathbb{N}, \quad 1 \leq u_i \leq n, \quad \forall i \in V \setminus \{0\}$$

Où $u_i$ représente la position du client $i$ dans sa tournée (pour éliminer les sous-tournées).

<u>4.2 Représentation heuristique (notre implémentation)</u>

Dans notre approche **métaheuristique ALNS**, la solution est représentée par une **structure de données par chemins** :

```python
routes: List[List[int]]  # Liste de routes (tournées)
```

**Exemple :**
```python
routes = [[1, 5, 3], [2, 7, 9], [4, 6, 8]]
```
- Route 1 : Dépôt → Client 1 → Client 5 → Client 3 → Dépôt
- Route 2 : Dépôt → Client 2 → Client 7 → Client 9 → Dépôt
- Route 3 : Dépôt → Client 4 → Client 6 → Client 8 → Dépôt

Cette représentation est **mathématiquement équivalente** aux variables $x_{ijk}$ mais plus adaptée aux algorithmes heuristiques.


## 5. Fonction Objectif

<u>5.1 Formulation exacte</u>

**Minimiser le temps total de parcours** (makespan - temps du dernier camion à terminer sa tournée) :

$$\min Z = \sum_{k \in K} \sum_{i \in V} \sum_{j \in V, j \neq i} d_{ij} \cdot x_{ijk}$$

Où $d_{ij}$ représente le **temps de trajet** entre les nœuds $i$ et $j$.

<u>5.2 Formulation heuristique avec pénalisation (implémentation ALNS)</u>

Notre approche transforme les **contraintes dures en contraintes souples** via une fonction de pénalisation :

$$\min Z = D + \lambda_C \cdot V_C + \lambda_T \cdot V_T$$
- $D$ = Temps total de parcours de toutes les tournées
Où :
- $D$ = Distance totale réelle des tournées
- $V_C$ = Violations de **capacité** (dépassements cumulés)
- $V_T$ = Violations de **fenêtres temporelles** (retards cumulés)
- $\lambda_C, \lambda_T$ = Coefficients de pénalité (typiquement $= 10000$)

**Temps total de parcours :**

**Distance totale :**
$$D = \sum_{r \in \text{routes}} \left( d_{0,r_1} + \sum_{i=1}^{|r|-1} d_{r_i, r_{i+1}} + d_{r_{|r|}, 0} \right)$$

**Violations de capacité :**
$$V_C = \sum_{r \in \text{routes}} \max\left(0, \sum_{i \in r} q_i - Q\right)$$

**Violations temporelles :**
$$V_T = \sum_{i=1}^{n} \max(0, t_i - b_i)$$

**Justification :** Cette approche permet aux métaheuristiques d'explorer temporairement des solutions non-faisables, facilitant l'échappement des minima locaux et la convergence vers de bonnes solutions.

## 6. Contraintes du Problème

<u>6.1 Contraintes de couverture</u>

**Chaque client doit être visité exactement une fois :**
$$\sum_{k \in K} \sum_{i \in V, i \neq j} x_{ijk} = 1, \quad \forall j \in V \setminus \{0\}$$

**Implémentation :** Garantie par construction dans la structure `routes`. Chaque client $j \in \{1, ..., n\}$ apparaît dans exactement une route.

<u>6.2 Contraintes de flot</u>

**Départ du dépôt :**
$$\sum_{j \in V \setminus \{0\}} x_{0jk} \leq 1, \quad \forall k \in K$$

**Retour au dépôt :**
$$\sum_{i \in V \setminus \{0\}} x_{i0k} \leq 1, \quad \forall k \in K$$

**Conservation du flot :**
$$\sum_{i \in V, i \neq j} x_{ijk} = \sum_{i \in V, i \neq j} x_{jik}, \quad \forall j \in V \setminus \{0\}, \forall k \in K$$

**Implémentation :** Implicitement respectées par la représentation en chemins. Chaque route commence au dépôt (0), visite une séquence de clients, et revient au dépôt (0).
# Pseudo-code du calcul du temps total d'une route
temps_total = d[0][route[0]]  # Temps: Dépôt → premier client
# Pseudo-code du calcul de distance d'une route
    temps_total += d[route[i]][route[i+1]]  # Temps: Client i → Client i+1
temps_total += d[route[-1]][0]  # Temps: Dernier client → Dépôt
    distance += d[route[i]][route[i+1]]  # Client i → Client i+1
distance += d[route[-1]][0]  # Dernier client → Dépôt
```

<u>6.3 Contraintes de capacité</u>

**Capacité maximale par tournée :**
$$\sum_{i \in V \setminus \{0\}} q_i \cdot y_{ik} \leq Q, \quad \forall k \in K$$

Où $y_{ik} = 1$ si le client $i$ est visité par le véhicule $k$, 0 sinon.

**Formulation équivalente :**
$$\sum_{i \in V \setminus \{0\}} q_i \cdot \sum_{j \in V, j \neq i} x_{ijk} \leq Q, \quad \forall k \in K$$

**Implémentation :**
```python
def _check_capacity_violation(route: List[int]) -> float:
    total_demand = sum(instance.customers[c].demand for c in route)
    if total_demand > instance.vehicle_capacity:
        return total_demand - instance.vehicle_capacity  # Violation
    return 0.0  # Faisable
```

<u>6.4 Contraintes de fenêtres temporelles</u>

**Respect des fenêtres temporelles :**
$$a_i \leq t_i \leq b_i, \quad \forall i \in V$$

**Cohérence temporelle (contrainte Big-M) :**
$$t_j \geq t_i + s_i + d_{ij} - M(1 - x_{ijk}), \quad \forall i, j \in V, \forall k \in K$$

Où $M$ est une constante suffisamment grande. Cette contrainte assure que si un véhicule se déplace de $i$ vers $j$ :
$$t_j \geq t_i + s_i + d_{ij}$$

**Contrainte stricte (pas d'attente) :**

Le véhicule doit arriver exactement dans la fenêtre temporelle, sans possibilité d'attendre :
$$a_i \leq t_i \leq b_i$$

Le temps d'arrivée au client suivant est :
$$t_{i+1} = t_i + s_i + d_{i,i+1}$$

**Implémentation :**
```python
def _check_time_violation(route: List[int]) -> float:
    current_time = 0.0
    violation = 0.0
    prev_customer = 0  # Dépôt
    
    for customer_id in route:
        customer = instance.customers[customer_id]
        
        # Calcul de l'heure d'arrivée
        travel_time = instance.distance(prev_customer, customer_id)
        arrival_time = current_time + travel_time
        
        # Vérification fenêtre temporelle (SANS attente)
        if arrival_time < customer.ready_time:
            # Arrivée trop tôt → VIOLATION
            violation += (customer.ready_time - arrival_time)
            current_time = arrival_time
        elif arrival_time > customer.due_time:
            # Arrivée trop tard → VIOLATION
            violation += (arrival_time - customer.due_time)
            current_time = arrival_time
        else:
            # Dans la fenêtre → OK
            current_time = arrival_time
        
        # Ajout du temps de service
        current_time += customer.service_time
        prev_customer = customer_id
    
    return violation
```

<u>6.5 Élimination des sous-tournées (MTZ)</u>

**Contraintes de Miller-Tucker-Zemlin :**
$$u_i - u_j + n \cdot x_{ijk} \leq n - 1, \quad \forall i, j \in V \setminus \{0\}, i \neq j, \forall k \in K$$

Où $u_i$ représente la position du client $i$ dans sa tournée.

**Implémentation :** Cette contrainte est **automatiquement satisfaite** par la représentation en chemins. Une route est par définition une séquence ordonnée de clients.

## 7. Modèle Complet - Formulation Mathématique

### Fonction Objectif
**Minimiser** le temps total de parcours (ou makespan) :
$$\min Z = \sum_{k \in K} \sum_{i \in V} \sum_{j \in V, j \neq i} d_{ij} \cdot x_{ijk}$$

### Sous contraintes

**1. Couverture des clients :**
$$\sum_{k \in K} \sum_{i \in V, i \neq j} x_{ijk} = 1, \quad \forall j \in V \setminus \{0\}$$

**2. Départ et retour au dépôt :**
$$\sum_{j \in V \setminus \{0\}} x_{0jk} \leq 1, \quad \forall k \in K$$
$$\sum_{i \in V \setminus \{0\}} x_{i0k} \leq 1, \quad \forall k \in K$$

**3. Conservation du flot :**
$$\sum_{i \in V, i \neq j} x_{ijk} = \sum_{i \in V, i \neq j} x_{jik}, \quad \forall j \in V \setminus \{0\}, \forall k \in K$$

**4. Capacité des véhicules :**
$$\sum_{i \in V \setminus \{0\}} q_i \cdot \sum_{j \in V, j \neq i} x_{ijk} \leq Q, \quad \forall k \in K$$

**5. Fenêtres temporelles :**
$$a_i \leq t_i \leq b_i, \quad \forall i \in V$$

**6. Cohérence temporelle :**
$$t_j \geq t_i + s_i + d_{ij} - M(1 - x_{ijk}), \quad \forall i, j \in V, \forall k \in K$$

**7. Élimination des sous-tournées (MTZ) :**
$$u_i - u_j + n \cdot x_{ijk} \leq n - 1, \quad \forall i, j \in V \setminus \{0\}, i \neq j, \forall k \in K$$

**8. Domaines des variables :**
$$x_{ijk} \in \{0, 1\}, \quad t_i \geq 0, \quad 1 \leq u_i \leq n$$
$$\forall i, j \in V, \forall k \in K$$

## 8. Formalisme de la Métaheuristique ALNS

<u>8.1 Espace de recherche</u>

**Espace des solutions :**
$$S = \{s = (r_1, r_2, ..., r_K) : \text{chaque } r_k \text{ est une séquence de clients}\}$$

Où chaque route $r_k$ satisfait (ou viole avec pénalité) :
- Capacité : $\sum_{i \in r_k} q_i \leq Q$
- Fenêtres temporelles : $a_i \leq t_i \leq b_i, \forall i \in r_k$

**Solution faisable :**
$$S_{\text{feas}} = \{s \in S : V_C(s) = 0 \text{ et } V_T(s) = 0\}$$

<u>8.2 Fonction d'évaluation</u>

$$f(s) = D(s) + \lambda_C \cdot V_C(s) + \lambda_T \cdot V_T(s)$$
- $D(s)$ : Temps total de parcours de la solution $s$
Avec :
- $D(s)$ : Distance totale de la solution $s$
- $V_C(s)$ : Violations de capacité
- $V_T(s)$ : Violations temporelles
- $\lambda_C, \lambda_T$ : Poids de pénalité

<u>8.3 Opérateurs de voisinage</u>

#### Voisinage local (Recherche Locale)

**1. 2-opt intra-route :**
$$N_{2\text{-opt}}(s) = \{s' : s' \text{ obtenue en inversant un segment d'une route}\}$$

**2. Relocate :**
$$N_{\text{relocate}}(s) = \{s' : s' \text{ obtenue en déplaçant un client vers une autre position}\}$$

**3. Swap :**
$$N_{\text{swap}}(s) = \{s' : s' \text{ obtenue en échangeant deux clients}\}$$

#### Voisinage large (ALNS)

**Opérateurs de destruction $D_i$** : $i \in \{1, 2, 3\}$
- $D_1$ : **Random Removal** - Retire $q$ clients aléatoirement
- $D_2$ : **Worst Removal** - Retire les $q$ clients les plus coûteux
- $D_3$ : **Shaw Removal** - Retire $q$ clients similaires (proximité spatiale et temporelle)

**Opérateurs de réparation $R_j$** : $j \in \{1, 2\}$
- $R_1$ : **Greedy Insertion** - Insère au coût minimal
- $R_2$ : **Regret-2 Insertion** - Priorise les clients difficiles à placer

**Voisinage ALNS :**
$$N_{\text{ALNS}}(s) = \bigcup_{i,j} R_j(D_i(s))$$

<u>8.4 Mécanisme de sélection adaptative</u>

**Poids des opérateurs :**
$$w_i^{(t+1)} = (1 - \rho) \cdot w_i^{(t)} + \rho \cdot \frac{\pi_i^{(t)}}{n_i^{(t)}}$$

Où :
- $w_i^{(t)}$ : Poids de l'opérateur $i$ à l'itération $t$
- $\rho$ : Taux d'apprentissage (typiquement $0.1$)
- $\pi_i^{(t)}$ : Score cumulé de l'opérateur $i$
- $n_i^{(t)}$ : Nombre d'utilisations de l'opérateur $i$

**Probabilité de sélection (roulette wheel) :**
$$P_i = \frac{w_i}{\sum_j w_j}$$

**Scores attribués :**
- Nouvelle meilleure solution globale : $\sigma_1 = 10$
- Amélioration de la solution courante : $\sigma_2 = 5$
- Solution acceptée (SA) : $\sigma_3 = 1$

<u>8.5 Critère d'acceptation (Simulated Annealing)</u>

Une solution $s'$ de coût $f(s')$ est acceptée avec probabilité :

$$P_{\text{accept}}(s \rightarrow s') = \begin{cases}
1 & \text{si } f(s') \leq f(s) \\
e^{-\frac{f(s') - f(s)}{T}} & \text{sinon}
\end{cases}$$

Où $T$ est la **température** qui décroît selon :
$$T^{(t+1)} = \alpha \cdot T^{(t)}$$

Avec :
- $T^{(0)}$ : Température initiale (typiquement $10000$)
- $\alpha$ : Taux de refroidissement (typiquement $0.9975$)

<u>8.6 Algorithme ALNS complet</u>

```
Algorithme ALNS(instance, max_iter, max_time)
Entrées : instance du CVRPTW, max_iter, max_time
Sortie : Meilleure solution trouvée

1. s_current ← ClarkeWright(instance)
2. s_best ← s_current
3. T ← T_init
4. w ← [1, 1, ..., 1]  // Poids initiaux
5. 
6. Pour t = 1 à max_iter faire :
7.     // Sélection adaptative des opérateurs
8.     i ← SelectDestroyOperator(w_destroy)
9.     j ← SelectRepairOperator(w_repair)
10.    
11.    // Destruction + Réparation
12.    s_temp ← s_current.copy()
13.    removed ← D_i(s_temp, q)
14.    R_j(s_temp, removed)
15.    
16.    // Évaluation
17.    Δ ← f(s_temp) - f(s_current)
18.    
19.    // Acceptation (Simulated Annealing)
20.    Si Δ ≤ 0 ou random() < exp(-Δ/T) alors :
21.        s_current ← s_temp
22.        
23.        // Mise à jour meilleure solution
24.        Si f(s_current) < f(s_best) alors :
25.            s_best ← s_current
26.            score ← σ_1
27.        Sinon si Δ < 0 alors :
28.            score ← σ_2
29.        Sinon :
30.            score ← σ_3
31.    Fin Si
32.    
33.    // Mise à jour poids adaptatifs
34.    w_i ← (1-ρ)·w_i + ρ·score
35.    
36.    // Refroidissement
37.    T ← α · T
38. Fin Pour
39. 
40. Retourner s_best
```


## 9. Complexité

<u>9.1 Complexité du problème</u>

Le CVRPTW est **NP-difficile** :
- Réduction depuis le TSP (NP-complet)
- Espace de recherche : $O((n!)^K)$ solutions possibles

<u>9.2 Complexité algorithmique</u>

**Clarke & Wright :**
- Calcul des économies : $O(n^2)$
- Tri : $O(n^2 \log n)$
- Fusion : $O(n^2)$
- **Complexité totale : $O(n^2 \log n)$**

**Recherche Locale (par itération) :**
- 2-opt : $O(n^2)$
- Relocate : $O(n^2)$
- Swap : $O(n^2)$

**ALNS (par itération) :**
- Destruction : $O(n)$
- Réparation : $O(n^2)$ (test de toutes les positions)
- **Complexité par itération : $O(n^2)$**

**Complexité globale ALNS :**
$$O(\text{max\_iter} \cdot n^2)$$

Pour $n = 1000$ clients et $1000$ itérations : environ $10^9$ opérations (quelques minutes de calcul).

<u>10. Correspondance Modèle ↔ Implémentation</u>

| Concept Mathématique | Implémentation Python | Fichier/Fonction |
|----------------------|----------------------|------------------|
| Temps $d_{ij}$ | `distance(i, j)` | `VRPInstance.distance()` |
| Fonction $f(s)$ | `calculate_cost()` | `Solution._calculate_cost()` |
| Distance $d_{ij}$ | `distance(i, j)` | `VRPInstance.distance()` |
| Demande $q_i$ | `customer.demand` | `Customer.demand` |
| Fenêtre $[a_i, b_i]$ | `[ready_time, due_time]` | `Customer.ready_time/due_time` |
| Capacité $Q$ | `vehicle_capacity` | `VRPInstance.vehicle_capacity` |
| Violation $V_C$ | `capacity_violations` | `Solution._check_capacity_violation()` |
| Violation $V_T$ | `time_violations` | `Solution._check_time_violation()` |
| Opérateur $D_1$ | `random_removal()` | Destruction operators |
| Opérateur $D_2$ | `worst_removal()` | Destruction operators |
| Opérateur $D_3$ | `shaw_removal()` | Destruction operators |
| Opérateur $R_1$ | `greedy_insertion()` | Repair operators |
| Opérateur $R_2$ | `regret_insertion()` | Repair operators |
| Critère SA | `accept(Δ, T)` | `ALNS.solve()` |
| Poids $w_i$ | `destroy_weights, repair_weights` | `ALNS.__init__()` |




# Méthode de Résolution Choisie
Dans le cadre de la résolution du CVRPTW, un problème NP-difficile encore plus complexe que le VRP classique à cause des fenêtres temporelles, le choix de la métaheuristique est un élément central.
Notre stratégie repose sur une approche hybride en trois niveaux :

- Clarke & Wright Savings → génération d’une solution initiale faisable
- Recherche Locale (Local Search) → amélioration itérative
- ALNS hybride avec Recuit Simulé → optimisation avancée et exploration globale

Cette combinaison permet d’obtenir une solution à la fois rapide à construire, facile à améliorer, et surtout performante sur des instances allant jusqu’à plusieurs milliers de clients.

## 1. Méthode Clarke & Wright : Solution initiale

Clarke & Wright (méthode des “Savings”) est une heuristique constructive utilisée pour générer rapidement une solution initiale au CVRPTW.
L’idée centrale est de mesurer l’“économie” réalisée lorsque deux clients sont visités l’un après l’autre au lieu de faire deux trajets séparés depuis le dépôt.
Dans le contexte du CVRPTW, Clarke & Wright fournit une base structurée, qui respecte généralement les premières contraintes temporelles, mais sans être optimale.
Cette solution initiale est indispensable pour lancer les phases d’amélioration ultérieures.


## 2. Recherche Local (Local search)

La solution issue de Clarke & Wright est ensuite améliorée par une recherche locale, basée sur l’exploration de plusieurs voisinages.
Voisinages utilisés :
Les déplacements élémentaires testés sont :

- 2-opt : inverser un segment d’une route
- Relocate : déplacer un client d’une route à une autre
- Swap : échanger deux clients entre deux routes

Ces voisinages permettent d’éliminer les détours inutiles et de rééquilibrer les tournées.

## 3. ALNS hybride Recuit Simulé

L’Adaptive Large Neighborhood Search (ALNS) constitue le cœur de notre méthode.
Il utilise une logique détruire / reconstruire et sélectionne dynamiquement les meilleurs opérateurs au cours du temps
Nous l’avons enrichi avec un mécanisme de Recuit Simulé (SA) pour accepter temporairement des solutions moins bonnes et éviter les minima locaux.

**Opérateurs de Destruction**

Une partie de la solution (10–40% des clients) est supprimée selon différents opérateurs :

- Random Removal : suppression aléatoire
- Shaw Removal : suppression de clients “proches” (distance, temps, TW)
- Worst-Removal : supprime les clients contribuant le plus au coût
- Time-Window Removal : supprime les clients causant retards / violations
- Capacity-Based Removal : spécifique CVRPTW, supprime les clients causant surcharge

Ces opérateurs permettent de casser la structure de la solution et d’explorer de nouvelles zones.

**Opérateurs de Reconstruction**

Les clients supprimés sont réinsérés via :

- Regret-2 / Regret-3 : insertion optimale en anticipant les choix futurs
- Best Insertion : insertion au coût minimal
- Time-Feasible Insertion : insertion respectant strictement les TW   
   
**Mécanisme d’acceptation – Recuit Simulé**

Une solution moins bonne peut être acceptée avec une probabilité :

                    P=e−Δ/T


Ce mécanisme permet :

- d’éviter les minima locaux,
- de favoriser la diversification,
- d’explorer des configurations non accessibles via Local Search.

La température décroît progressivement selon :

                    Tk+1​=αTk​

**Adaptation dynamique**

ALNS attribue un score à chaque opérateur (destruction / reconstruction).
Les opérateurs les plus performants deviennent plus probables au fil du temps.
C’est une forme de méta-apprentissage interne, permettant à l’algorithme d’adapter son comportement à la structure de l’instance.

**Formalisation du Voisinage dans la Métaheuristique**

Dans notre approche :
Le voisinage local est défini par les mouvements :

- {2-opt, relocate, swap, or-opt}

Le voisinage large (Large Neighborhood) est défini par la paire :
            (operateur de destruction, operateur de reparation)

Le croisement dans l’ALNS correspond implicitement à la reconstruction de segments entiers de tournées.

Le voisinage total est donc très vaste, ce qui améliore considérablement la capacité d’exploration de l’espace des solutions.	​


# Implémentation

L’implémentation a été réalisée en Python dans un environnement Jupyter Notebook, en suivant une architecture modulaire.
Chaque composant (construction, voisinage, destruction/réparation, évaluation…) est isolé afin de faciliter les tests et l’intégration progressive.

Toutes les fonctions associées aux trois étapes principales — Clarke & Wright, Recherche Locale, et ALNS hybride Recuit Simulé — sont regroupées dans ce fichier, chacune accompagnée de commentaires détaillés.

L’objectif était de séparer :

- la logique algorithmique (dans le fichier Python),
- et la partie analyse / expérimentation (dans le notebook .ipynb).

Ainsi, le fichier d’implémentation contient :

- une description en commentaire de chaque étape de l’algorithme,
- les détails des paramètres utilisés,
- les règles de faisabilité appliquées lors des opérations (capacité, fenêtres temporelles, temps de service),
- et des explications sur le fonctionnement de chaque opérateur (voisinages, destructions, réparations, acceptation SA).

Enfin, tout le code a été documenté de manière à rendre le comportement de chaque fonction explicite, ce qui permet d’avoir une implémentation non seulement fonctionnelle mais également pédagogique.

# Etude expérimentale

# Conclusion

Dans le cadre de l'appel à manifestation d'intérêt de l'ADEME pour l'optimisation de la mobilité et de la logistique, notre équipe a développé une solution complète pour le problème de tournées de véhicules avec fenêtres temporelles et contraintes de capacité (CVRPTW).

Nous avons d'abord établi une **modélisation mathématique rigoureuse** du problème, en formalisant les variables de décision, la fonction objectif et l'ensemble des contraintes (couverture des clients, flot, capacité, fenêtres temporelles). Cette étape nous a permis de comprendre la complexité NP-difficile du problème et de justifier le recours à des métaheuristiques.

Notre **approche de résolution hybride** combine trois niveaux d'optimisation :
- **Clarke & Wright** pour générer rapidement une solution initiale faisable
- **Recherche Locale** (2-opt, relocate, swap) pour améliorer la solution
- **ALNS avec Recuit Simulé** pour explorer efficacement l'espace de recherche

Cette stratégie permet de traiter des instances de grande taille (jusqu'à plusieurs milliers de clients) en obtenant des solutions de bonne qualité en un temps raisonnable.

L'**implémentation en Python** a été réalisée de manière modulaire et documentée, facilitant les tests et l'extension future du code. L'étude expérimentale valide la performance de notre algorithme sur des instances de référence.

Ce projet démontre que **l'optimisation par recherche opérationnelle constitue un levier efficace pour réduire l'empreinte carbone** du secteur logistique. En minimisant les distances parcourues tout en respectant les contraintes opérationnelles, notre solution contribue directement aux objectifs environnementaux de l'ADEME.

**Perspectives d'amélioration :**
- Intégration de flottes hétérogènes et multi-dépôts
- Prise en compte du trafic dynamique en temps réel
- Optimisation multi-objectifs (distance, nombre de véhicules, émissions CO₂)
- Extension aux véhicules électriques avec contraintes de recharge

En conclusion, ce travail illustre comment une approche scientifique rigoureuse, alliant modélisation mathématique et algorithmes performants, peut apporter des réponses concrètes aux défis environnementaux et logistiques actuels.
